In [ ]:
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor

from Funcoes_Comuns import avaliar_modelo, registrar_modelo

In [2]:
# Obter dados
df_enem = pd.read_pickle('Bases\MICRODADOS_ENEM_2023_tratados.pkl')

Label Encoding: apenas transforma variáveis categóricas em numéricas, não há significado numérico

- Processamento necessário para modelos aplicados: random forest baseado em árvore de decisão

In [3]:
# Aplicar o LabelEncoder para converter as colunas categóricas em numéricas
label_encoder = LabelEncoder()
categorical_columns = df_enem.select_dtypes(include=['category']).columns

for col in categorical_columns:
    df_enem[col] = label_encoder.fit_transform(df_enem[col])

In [4]:
df_enem.head()

,BIN_Q001_DUMMY_H,BIN_Q002_DUMMY_H,BIN_Q018,BIN_Q020,BIN_Q021,BIN_Q023,BIN_Q025,CAT_COR_RACA,CAT_CO_MUNICIPIO_ESC,CAT_CO_UF_ESC,...,NUM_Q011,NUM_Q012,NUM_Q013,NUM_Q014,NUM_Q015,NUM_Q016,NUM_Q017,NUM_Q019,NUM_Q022,NUM_Q024
0,False,False,False,False,False,False,True,3,941,9,...,0,1,0,0,0,0,0,1,3,0
1,False,False,False,False,False,False,False,3,1031,9,...,0,1,0,0,0,0,0,1,1,0
2,False,False,False,False,False,False,True,3,4232,20,...,0,1,0,1,0,0,0,1,1,1
3,False,False,False,False,False,True,True,1,169,4,...,0,1,0,0,0,0,0,1,3,0
4,False,False,False,False,False,False,True,1,3770,19,...,0,1,0,1,0,0,0,1,1,0


In [5]:
variaveis_alvo = ['NUM_NOTA_MT', 'NUM_NOTA_LC', 'NUM_NOTA_CN', 'NUM_NOTA_CH', 'NUM_NOTA_REDACAO']

# separar em treino e teste
X = df_enem.drop(columns=variaveis_alvo)
y = df_enem[variaveis_alvo]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Ajuste de tipo para MLflow
# Converter colunas inteiras para float
X_train = X_train.astype({col: 'float' for col in X_train.select_dtypes('int').columns})
X_test = X_test.astype({col: 'float' for col in X_test.select_dtypes('int').columns})

Modelo Base Random Forest

In [ ]:
len(X_train)
amostra = 50000  # Até 50 mil o processamento é rápido: aproximadamente 8 minutos com 200 árvores

    Utilizar amostras menores para testes rápidos
    max_features como None deixa mais lento o treinamento

In [79]:
parametros = {
    'n_estimators': 400,
    'max_depth': 50,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'bootstrap': True,
    'criterion': 'absolute_error',
    'random_state': 42,
    'verbose': 2,
    'n_jobs': -1,
}

In [80]:
# Treinar o modelo
modelo_random_forest = RandomForestRegressor(**parametros)
modelo_random_forest.fit(X_train[:amostra], y_train['NUM_NOTA_CH'][:amostra])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 400
building tree 2 of 400
building tree 3 of 400
building tree 4 of 400
building tree 5 of 400
building tree 6 of 400
building tree 7 of 400
building tree 8 of 400
building tree 9 of 400
building tree 10 of 400
building tree 11 of 400
building tree 12 of 400
building tree 13 of 400
building tree 14 of 400
building tree 15 of 400
building tree 16 of 400
building tree 17 of 400
building tree 18 of 400
building tree 19 of 400
building tree 20 of 400
building tree 21 of 400
building tree 22 of 400
building tree 23 of 400
building tree 24 of 400
building tree 25 of 400
building tree 26 of 400
building tree 27 of 400
building tree 28 of 400
building tree 29 of 400
building tree 30 of 400
building tree 31 of 400
building tree 32 of 400
building tree 33 of 400


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.9min


building tree 34 of 400
building tree 35 of 400
building tree 36 of 400
building tree 37 of 400
building tree 38 of 400
building tree 39 of 400
building tree 40 of 400
building tree 41 of 400
building tree 42 of 400
building tree 43 of 400
building tree 44 of 400
building tree 45 of 400
building tree 46 of 400
building tree 47 of 400
building tree 48 of 400
building tree 49 of 400
building tree 50 of 400
building tree 51 of 400
building tree 52 of 400
building tree 53 of 400
building tree 54 of 400
building tree 55 of 400
building tree 56 of 400
building tree 57 of 400
building tree 58 of 400
building tree 59 of 400
building tree 60 of 400
building tree 61 of 400
building tree 62 of 400
building tree 63 of 400
building tree 64 of 400
building tree 65 of 400
building tree 66 of 400
building tree 67 of 400
building tree 68 of 400
building tree 69 of 400
building tree 70 of 400
building tree 71 of 400
building tree 72 of 400
building tree 73 of 400
building tree 74 of 400
building tree 75

[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 10.6min


building tree 155 of 400
building tree 156 of 400
building tree 157 of 400
building tree 158 of 400
building tree 159 of 400
building tree 160 of 400
building tree 161 of 400
building tree 162 of 400
building tree 163 of 400
building tree 164 of 400
building tree 165 of 400
building tree 166 of 400
building tree 167 of 400
building tree 168 of 400
building tree 169 of 400
building tree 170 of 400
building tree 171 of 400
building tree 172 of 400
building tree 173 of 400
building tree 174 of 400
building tree 175 of 400
building tree 176 of 400
building tree 177 of 400
building tree 178 of 400
building tree 179 of 400
building tree 180 of 400
building tree 181 of 400
building tree 182 of 400
building tree 183 of 400
building tree 184 of 400
building tree 185 of 400
building tree 186 of 400
building tree 187 of 400
building tree 188 of 400
building tree 189 of 400
building tree 190 of 400
building tree 191 of 400
building tree 192 of 400
building tree 193 of 400
building tree 194 of 400


[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 25.1min


building tree 358 of 400
building tree 359 of 400
building tree 360 of 400
building tree 361 of 400
building tree 362 of 400
building tree 363 of 400
building tree 364 of 400
building tree 365 of 400
building tree 366 of 400
building tree 367 of 400
building tree 368 of 400
building tree 369 of 400
building tree 370 of 400
building tree 371 of 400
building tree 372 of 400
building tree 373 of 400
building tree 374 of 400
building tree 375 of 400
building tree 376 of 400
building tree 377 of 400
building tree 378 of 400
building tree 379 of 400
building tree 380 of 400
building tree 381 of 400
building tree 382 of 400
building tree 383 of 400
building tree 384 of 400
building tree 385 of 400
building tree 386 of 400
building tree 387 of 400
building tree 388 of 400
building tree 389 of 400
building tree 390 of 400
building tree 391 of 400
building tree 392 of 400
building tree 393 of 400
building tree 394 of 400
building tree 395 of 400
building tree 396 of 400
building tree 397 of 400


[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 28.5min finished


RandomForestRegressor(criterion='absolute_error', max_depth=50,
                      max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=5, n_estimators=400, n_jobs=-1,
                      random_state=42, verbose=2)

In [82]:
# Avaliação do modelo final
y_pred_rf = modelo_random_forest.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    7.7s
[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:    8.8s finished


In [83]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento, 
                     parametros={**parametros, 'amostra': amostra}, 
                     X_train=X_train[:amostra], 
                     y_train=y_train[:amostra], 
                     y_test=y_test, 
                     y_pred=y_pred_rf, 
                     varivel_alvo='NUM_NOTA_CH', 
                     modelo=modelo_random_forest, 
                     nome_modelo='modelo_random_forest',
                     descricao_modelo='Modelo Random Forest')

Successfully registered model 'modelo_random_forest'.
2025/05/20 21:01:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_random_forest, version 1
Created version '1' of model 'modelo_random_forest'.


🏃 View run capricious-bird-157 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/f4ba0e1d3eb94826a318ff98b4ee54d0
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_random_forest
Rastreamento do MLflow finalizado.


In [84]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_random_forest.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_rf, "teste")

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    9.3s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:   24.7s


MAE (treino): 54.43
RMSE (treino): 69.45
R2 (treino): 0.33
MAE (teste): 56.07
RMSE (teste): 71.12
R2 (teste): 0.29


[Parallel(n_jobs=8)]: Done 400 out of 400 | elapsed:   27.9s finished


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_random_forest, 'Modelos\modelo_random_forest_base.pkl')

['Modelos\\modelo_reandom_forest_base.pkl']

Aplicar Random Search

In [119]:
len(X_train)
amostra = 10000

In [121]:
param_distributions = {
    'n_estimators': [100, 300, 500],                  # Número de árvores
    'max_depth': range(50, 100, 10),                  # Profundidade máxima das árvores
    'min_samples_split': [2, 5, 10],                  # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 5],                    # Número mínimo de amostras em uma folha
    'max_features': ['sqrt', 'log2'],                 # Número de features consideradas para divisão
    'bootstrap': [True],                              # Usar bootstrap ou não
    'criterion': ['absolute_error'],                  # Critério de divisão
    'ccp_alpha': [0.2, 0.3436, 0.4]                   # Parâmetro de poda de complexidade de custo
}

In [122]:
# Configurar o RandomizedSearchCV: este métodos testa combinações aleatórias de hiperparâmetros

n_iter = 5  # Número de combinações a serem testadas
kf = KFold(n_splits=4, shuffle=True, random_state=42)  # Divisão em partes para validação cruzada

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=kf,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=-1,
    random_state=42
)

In [123]:
# Executar o Random Search
random_search.fit(X_train[:amostra], y_train['NUM_NOTA_CH'][:amostra])

Fitting 4 folds for each of 5 candidates, totalling 20 fits


RandomizedSearchCV(cv=KFold(n_splits=4, random_state=42, shuffle=True),
                   estimator=RandomForestRegressor(random_state=42), n_iter=5,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'ccp_alpha': [0.2, 0.3436, 0.4],
                                        'criterion': ['absolute_error'],
                                        'max_depth': range(50, 100, 10),
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 5],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 300, 500]},
                   random_state=42, scoring='neg_mean_absolute_error',
                   verbose=2)

In [124]:
# Melhores parâmetros encontrados
best_params = random_search.best_params_
print("Melhores parâmetros encontrados: ", best_params)

Melhores parâmetros encontrados:  {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 60, 'criterion': 'absolute_error', 'ccp_alpha': 0.2, 'bootstrap': True}


In [125]:
# Treinar o modelo com os melhores parâmetros
modelo_random_forest_random_search = RandomForestRegressor(**best_params, n_jobs=-1, random_state=42)
modelo_random_forest_random_search.fit(X_train[:amostra], y_train['NUM_NOTA_CH'][:amostra])

RandomForestRegressor(ccp_alpha=0.2, criterion='absolute_error', max_depth=60,
                      max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=10, n_estimators=500, n_jobs=-1,
                      random_state=42)

In [127]:
# Avaliação do modelo final
y_pred_rf_rs = modelo_random_forest_random_search.predict(X_test)

In [128]:
nome_experimento = 'Notas CH ENEM 2023'

registrar_modelo(experimento=nome_experimento, 
                     parametros={**best_params, 'amostra': amostra}, 
                     X_train=X_train[:amostra], 
                     y_train=y_train[:amostra], 
                     y_test=y_test, 
                     y_pred=y_pred_rf_rs, 
                     varivel_alvo='NUM_NOTA_CH', 
                     modelo=modelo_random_forest_random_search, 
                     nome_modelo='modelo_random_forest_random_search',
                     descricao_modelo='Modelo Random Forest com RandomizedSearchCV')

Registered model 'modelo_random_forest_random_search' already exists. Creating a new version of this model...
2025/05/20 21:31:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_random_forest_random_search, version 19


🏃 View run clumsy-rook-387 at: http://127.0.0.1:9080/#/experiments/957135083854196683/runs/615e2a99fb704c1a94f2134a97048441
🧪 View experiment at: http://127.0.0.1:9080/#/experiments/957135083854196683
Modelo registrado com sucesso no MLflow: modelo_random_forest_random_search
Rastreamento do MLflow finalizado.


Created version '19' of model 'modelo_random_forest_random_search'.


In [129]:
# Avaliação grupo treino
avaliar_modelo(y_train['NUM_NOTA_CH'], modelo_random_forest_random_search.predict(X_train), "treino")

# Avaliação grupo teste
avaliar_modelo(y_test['NUM_NOTA_CH'], y_pred_rf_rs, "teste")

MAE (treino): 57.97
RMSE (treino): 73.68
R2 (treino): 0.24
MAE (teste): 57.97
RMSE (teste): 73.69
R2 (teste): 0.24


In [ ]:
# Salvar o modelo otimizado como um arquivo pickle
joblib.dump(modelo_random_forest_random_search, 'Modelos\modelo_random_forest_random_search.pkl')

['Modelos\\modelo_reandom_forest_random_search.pkl']